In [1]:
from statsmodels.compat.pandas import deprecate_kwarg
from statsmodels.tsa.arima_model import ARIMA
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
# additive decompose a contrived additive time series
from random import randrange
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings('ignore')

data = pd.read_csv('/Users/alket/Desktop/dati/new_data_backfill_forwfill.csv', index_col = 0, 
                   header=0, parse_dates=True)
agg_by_cell = data.groupby(by = ['cell_num'])
print(len(agg_by_cell), 'celle')

221 celle


In [2]:
# create a differenced series
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return np.array(diff)
 
# invert differenced value
def inverse_difference(history, yhat, interval=1):
    return yhat + history[-interval]

In [3]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequence)):
         # find the end of this pattern
         end_ix = i + n_steps_in
         out_end_ix = end_ix + n_steps_out
         # check if we are beyond the sequence
         if out_end_ix > len(sequence):
             break
         # gather input and output parts of the pattern
         seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
         X.append(seq_x)
         y.append(seq_y)
    return np.array(X), np.array(y)

In [4]:
def get_forcast(series, st_in, st_out):
    
    # split into samples
    X, y = split_sequence(series, st_in, st_out)
    
    days_in_year = 24
    count = 0
    result = []
    for i in range(len(X)):
        count += 1
        differenced = difference(X[i], days_in_year)
        #print(count)
        # fit model
        model = ARIMA(differenced, order=(8,0,1))
        model_fit = model.fit(disp=0)

        # multi-step out-of-sample forecast
        forecast = model_fit.forecast(steps=st_out)[0]

        # invert the differenced forecast to something usable
        history = [x for x in X[i]]
        day = 1
        
        predicted = []
        
        for yhat in forecast:
            inverted = inverse_difference(history, yhat, days_in_year)
            history.append(inverted)
            day += 1
            predicted.append(inverted)
        
        predicted = np.array(predicted)    
        result.append(predicted)
    return result

In [5]:
step_in, step_out = 2600, 24


In [11]:
import math

In [12]:
parameters = pd.read_csv('BestARIMA_config_parametres_V2.csv', header = None, sep = ':')
par_per_cell = {}
for index, row in parameters.iterrows(): 
    cell = row[0]
    rest = row[1]
    par = rest.split('),')
    f_s = par[0]
    fs = f_s[2:]
    par_per_cell[cell] = fs

In [17]:
predicted = []
count = 0
dict2data = {}
dict2MAPE = {}
dict2RMSE = {}
for i, k in agg_by_cell:
    predicted = []
    cell = i
    param_per_cell_i = par_per_cell[cell]
   
    print(count)
    count +=1
    #if count < 23: continue
    if count < 170: continue
    if count > 219: break    
    series_i = k.iloc[::4, :]
   
    series_i = series_i['nr_people'].values
   
    X_data, y_data = split_sequence(series_i, step_in, step_out)
    forcasted = get_forcast(series_i, step_in, step_out)
    error = abs(forcasted - y_data)
    pow_err = np.power(error, 2)
    rmse = math.sqrt(np.mean(pow_err))
    
    mape_i = np.mean(100 * np.divide(error, y_data))
    
    # collect data 2 dictionary
    minimum = np.amin(error)   
    per75 = np.percentile(error, 75)
    per50 = np.percentile(error, 50)
    per25 = np.percentile(error, 25)
    maximum = np.amax(error)
    l5i = [minimum, per25, per50, per75, maximum]
    #l5i = ','.join(l5i)
    dict2data[cell] = l5i
    dict2MAPE[cell] = mape_i
    dict2RMSE[cell] = rmse

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219


In [18]:
with open('MAE_error_data_4_ARIMA_MultistepForcast_3.3.csv', 'w') as f:
    for key, value in dict2data.items():
        f.write('%s:%s\n' % (key, value))
        
with open('MAPE_error_data_4_ARIMA_MultistepForcast_3.3.csv', 'w') as f:
    for key, value in dict2MAPE.items():
        f.write('%s:%s\n' % (key, value))  
        
with open('RMSE_error_data_4_ARIMA_MultistepForcast_3.3.csv', 'w') as f:
    for key, value in dict2RMSE.items():
        f.write('%s:%s\n' % (key, value))         